In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is not available


In [2]:
# Read from CSV instead of pickle
dataset = pd.read_csv('cleaned_profiles.csv')
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,NaN,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,NaN,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,NaN,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,NaN,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,NaN,"{'0': {'Role': 'Bartender', 'Workplace': ""O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,"{'Full Name': 'Emily Williams', 'Workplace': ""...",Emily Williams,Market Research Analyst at L'Oreal,"New York City, New York",106,No,717,{},Market Research Analyst Elizabeth Arden Jan 20...,3,...,0,{},0,{},0,{},3,{},4,11
3596,"{'Full Name': 'Sarah Johnson', 'Workplace': 'D...",Sarah Johnson,Director of Marketing Strategy at Acme Inc.,"San Francisco, California",102,No,6,{},Director of Marketing Acme Inc. January 2018-P...,5,...,0,{},0,{},0,{},5,{},3,11
3597,"{'Full Name': 'Sarah Johnson', 'Workplace': 'M...",Sarah Johnson,Manager at Acme Inc.,"New York City, New York",435,No,10,A results-driven and experienced professional ...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,"{'Full Name': 'Emma Williams', 'Workplace': 'M...",Emma Williams,Manager and Director at ABC Inc.,"Seattle, Washington",280,No,34,With over 10 years of experience in operations...,Operations Manager ABC Inc. January 2015- Pres...,1,...,0,English Spanish,2,{},0,{},4,{},4,11


In [ ]:
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def fun(index):
    # Initialize empty string to store all text
    all_text = ""

    # Process intro section
    intro = safe_eval(dataset["Intro"].iloc[index])
    if isinstance(intro, dict):
        all_text += f" {intro.get('Full Name', '')} "
        all_text += f" {intro.get('Workplace', '')} "
        all_text += f" {intro.get('Location', '')} "

    # Process about section
    about = safe_eval(dataset["About"].iloc[index])
    if isinstance(about, dict):
        all_text += f" {about.get('About', '')} "
    else:
        all_text += f" {str(about) if pd.notna(about) else ''} "

    # Process experiences section
    experiences = safe_eval(dataset["Experiences"].iloc[index])
    if isinstance(experiences, dict):
        for key, value in experiences.items():
            if isinstance(value, dict):
                all_text += f" {value.get('Role', '')} "
                all_text += f" {value.get('Workplace', '')} "
                all_text += f" {value.get('Duration', '')} "
                all_text += f" {value.get('Workplace Location', '')} "
                all_text += f" {value.get('Description', '')} "

    # Generate single RoBERTa embedding for all text
    embedding = get_roberta_embedding(all_text.strip())

    print(index)

    return embedding

# Function to generate embeddings for the dataset
def generate_embeddings_csv(dataset, output_path, num_rows=3600):
    all_embeddings = []

    for index in range(min(num_rows, len(dataset))):
        try:
            embedding = fun(index)
            all_embeddings.append(embedding)
            print(f"Processed row {index + 1}")
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            # RoBERTa base also has 768-dimensional embeddings
            all_embeddings.append(np.zeros(768))

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save to CSV
    embeddings_df.to_csv(output_path, index=False)
    print(f"Embeddings saved to {output_path}")

# Usage
output_path = 'roberta_raw_text_tokenised_NEW.csv'

# Generate embeddings for the first 3600 rows and save to CSV
generate_embeddings_csv(dataset, output_path, num_rows=3600)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Processed row 1101
1101
Processed row 1102
1102
Processed row 1103
1103
Processed row 1104
1104
Processed row 1105
1105
Processed row 1106
1106
Processed row 1107
1107
Processed row 1108
1108
Processed row 1109
1109
Processed row 1110
1110
Processed row 1111
1111
Processed row 1112
1112
Processed row 1113
1113
Processed row 1114
1114
Processed row 1115
1115
Processed row 1116
1116
Processed row 1117
1117
Processed row 1118
1118
Processed row 1119
1119
Processed row 1120
1120
Processed row 1121
1121
Processed row 1122
1122
Processed row 1123
1123
Processed row 1124
1124
Processed row 1125
1125
Processed row 1126
1126
Processed row 1127
1127
Processed row 1128
1128
Processed row 1129
1129
Processed row 1130
1130
Processed row 1131
1131
Processed row 1132
1132
Processed row 1133
1133
Processed row 1134
1134
Processed row 1135
1135
Processed row 1136
1136
Processed row 1137
1137
Processed row 1138
1138
Processed row 1139
1139
Processed row

In [ ]:
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def process_row(index):
    # Extract textual data
    intro = safe_eval(dataset["Intro"].iloc[index])
    full_name = dataset["Full Name"].iloc[index]
    workplace = dataset["Workplace"].iloc[index]
    location = dataset["Location"].iloc[index]
    about = dataset["About"].iloc[index]
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process textual data
    textual_data = f"{full_name} {workplace} {location} {about}"

    if isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                textual_data += f" {value.get('Role', '')} {value.get('Workplace', '')} {value.get('Duration', '')} {value.get('Workplace Location', '')} {value.get('Description', '')}"

    # Extract numerical fields
    numerical_fields = [
        "Number of Experiences", "Number of Educations", "Number of Licenses",
        "Number of Volunteering", "Number of Skills", "Number of Recommendations",
        "Number of Projects", "Number of Publications", "Number of Courses",
        "Number of Honors", "Number of Scores", "Number of Languages",
        "Number of Organizations", "Number of Interests", "Number of Activities",
        "Connections", "Followers"
    ]

    numerical_data = []
    for field in numerical_fields:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)  # Default to 0 if conversion fails

    # Concatenate textual and numerical data
    combined_data = f"{textual_data} {' '.join(map(str, numerical_data))}"

    # Generate RoBERTa embedding for combined data
    combined_embedding = get_roberta_embedding(combined_data)

    print(index)

    return combined_embedding

def generate_embeddings_csv(dataset, output_path, num_rows=3600):
    all_embeddings = []

    for index in range(min(num_rows, len(dataset))):
        try:
            embedding = process_row(index)
            all_embeddings.append(embedding)
            print(f"Processed row {index + 1}")
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            # RoBERTa embeddings are 768-dimensional like BERT
            all_embeddings.append(np.zeros(768))

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save to CSV
    embeddings_df.to_csv(output_path, index=False)
    print(f"Embeddings saved to {output_path}")

# Usage
output_path = 'roberta_concatenated_tokenised_noSSTE_NEW.csv'

# Generate embeddings for the first 3600 rows and save to CSV
generate_embeddings_csv(dataset, output_path, num_rows=3600)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Processed row 1101
1101
Processed row 1102
1102
Processed row 1103
1103
Processed row 1104
1104
Processed row 1105
1105
Processed row 1106
1106
Processed row 1107
1107
Processed row 1108
1108
Processed row 1109
1109
Processed row 1110
1110
Processed row 1111
1111
Processed row 1112
1112
Processed row 1113
1113
Processed row 1114
1114
Processed row 1115
1115
Processed row 1116
1116
Processed row 1117
1117
Processed row 1118
1118
Processed row 1119
1119
Processed row 1120
1120
Processed row 1121
1121
Processed row 1122
1122
Processed row 1123
1123
Processed row 1124
1124
Processed row 1125
1125
Processed row 1126
1126
Processed row 1127
1127
Processed row 1128
1128
Processed row 1129
1129
Processed row 1130
1130
Processed row 1131
1131
Processed row 1132
1132
Processed row 1133
1133
Processed row 1134
1134
Processed row 1135
1135
Processed row 1136
1136
Processed row 1137
1137
Processed row 1138
1138
Processed row 1139
1139
Processed row

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaModel
import logging
import time
from tqdm import tqdm

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("embeddings_log.log"),
        logging.StreamHandler()
    ]
)

# Check GPU availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base').to(device)
model.eval()

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Move input tensors to GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Move output back to CPU for numpy conversion
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def fun(index):
    start_time = time.time()

    # Handle 'Intro' column
    row_intro = dataset.iloc[index]['Intro']
    if isinstance(row_intro, dict):
        full_name = row_intro.get("Full Name", "")
        workplace = row_intro.get("Workplace", "")
        location = row_intro.get("Location", "")
    elif isinstance(row_intro, str):
        try:
            row_intro = eval(row_intro)
            full_name = row_intro.get("Full Name", "")
            workplace = row_intro.get("Workplace", "")
            location = row_intro.get("Location", "")
        except:
            full_name = workplace = location = ""
    else:
        full_name = workplace = location = ""

    # Handle 'About' column
    about = dataset.iloc[index]['About']
    if isinstance(about, dict):
        about = about.get("About", "")
    elif isinstance(about, str):
        try:
            about_dict = eval(about)
            about = about_dict.get("About", "")
        except:
            pass
    about = str(about)

    # Handle 'Experiences' column
    row_exp = dataset.iloc[index]['Experiences']
    roles, workplaces_exp, durations, workplace_locations, descriptions = [], [], [], [], []
    if isinstance(row_exp, dict):
        for key, value in row_exp.items():
            if isinstance(value, dict):
                roles.append(value.get("Role", ""))
                workplaces_exp.append(value.get("Workplace", ""))
                durations.append(value.get("Duration", ""))
                workplace_locations.append(value.get("Workplace Location", ""))
                descriptions.append(value.get("Description", ""))
    elif isinstance(row_exp, str):
        try:
            row_exp = eval(row_exp)
            for key, value in row_exp.items():
                if isinstance(value, dict):
                    roles.append(value.get("Role", ""))
                    workplaces_exp.append(value.get("Workplace", ""))
                    durations.append(value.get("Duration", ""))
                    workplace_locations.append(value.get("Workplace Location", ""))
                    descriptions.append(value.get("Description", ""))
        except:
            roles, workplaces_exp, durations, workplace_locations, descriptions = [""], [""], [""], [""], [""]
    else:
        roles, workplaces_exp, durations, workplace_locations, descriptions = [""], [""], [""], [""], [""]

    try:
        # Generate RoBERTa embeddings
        workplace_emb = get_roberta_embedding(workplace)
        location_emb = get_roberta_embedding(location)
        about_emb = get_roberta_embedding(about)
        workplaces_exp_emb = get_roberta_embedding(" ".join(workplaces_exp))
        roles_emb = get_roberta_embedding(" ".join(roles))
        durations_emb = get_roberta_embedding(" ".join(durations))
        workplace_locations_emb = get_roberta_embedding(" ".join(workplace_locations))
        descriptions_emb = get_roberta_embedding(" ".join(descriptions))

        # Generate tag embeddings
        intro_tag = get_roberta_embedding("Introduction")
        workplace_tag = get_roberta_embedding("workplace")
        location_tag = get_roberta_embedding("location")
        about_tag = get_roberta_embedding("About")
        desc_tag = get_roberta_embedding("description")
        exp_tag = get_roberta_embedding("Experiences")
        role_tag = get_roberta_embedding("role")
        duration_tag = get_roberta_embedding("duration")

        # Calculate final embeddings
        fin1 = workplace_emb - (intro_tag + workplace_tag) / 2
        fin2 = location_emb - (intro_tag + location_tag) / 2
        fin3 = about_emb - (about_tag + desc_tag) / 2
        fin4 = workplaces_exp_emb - (exp_tag + workplace_tag) / 2
        fin5 = roles_emb - (exp_tag + role_tag) / 2
        fin6 = durations_emb - (exp_tag + duration_tag) / 2
        fin7 = workplace_locations_emb - (exp_tag + location_tag) / 2
        fin8 = descriptions_emb - (exp_tag + desc_tag) / 2

        fin = (fin1 + fin2 + fin3 + fin4 + fin5 + fin6 + fin7 + fin8) / 8

    except RuntimeError as e:
        if "out of memory" in str(e):
            torch.cuda.empty_cache()
            print(f"GPU out of memory at row {index}. Clearing cache and returning zeros.")
            return np.zeros(768)
        raise e

    end_time = time.time()
    execution_time = end_time - start_time

    return fin

def generate_embeddings_csv(dataset, output_path, num_rows=3600):
    all_embeddings = []
    total_start_time = time.time()

    # Add progress bar
    for index in tqdm(range(min(num_rows, len(dataset))), desc="Processing rows"):
        try:
            embedding = fun(index)
            all_embeddings.append(embedding)

            # Clear GPU cache periodically
            if (index + 1) % 10 == 0:
                torch.cuda.empty_cache()

            # Save progress periodically
            if (index + 1) % 100 == 0:
                temp_df = pd.DataFrame(all_embeddings)
                temp_df.to_csv(f"{output_path}_temp_{index+1}.csv", index=False)
                print(f"\nSaved checkpoint at row {index+1}")

        except Exception as e:
            print(f"\nError processing row {index}: {str(e)}")
            all_embeddings.append(np.zeros(768))

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save to CSV
    embeddings_df.to_csv(output_path, index=False)

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"\nTotal execution time: {total_execution_time:.2f} seconds")
    print(f"Average time per row: {total_execution_time/len(all_embeddings):.2f} seconds")

# Usage
output_path = 'roberta_text_withSSTE_tokenised_NEW.csv'

# Generate embeddings for the first 3600 rows and save to CSV
generate_embeddings_csv(dataset, output_path, num_rows=3600)

Using device: cuda


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing rows:   3%|▎         | 100/3600 [00:23<12:45,  4.57it/s]


Saved checkpoint at row 100


Processing rows:   6%|▌         | 201/3600 [00:41<12:29,  4.54it/s]


Saved checkpoint at row 200


...

Processing rows:  97%|█████████▋| 3501/3600 [12:36<01:35,  1.04it/s]


Saved checkpoint at row 3500


Processing rows: 100%|██████████| 3600/3600 [13:00<00:00,  4.61it/s]


Saved checkpoint at row 3600



Total execution time: 784.18 seconds
Average time per row: 0.22 seconds


In [3]:
import logging
import time
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def process_subsection(text, section_tag, subsection_tag):
    tokens = tokenizer.tokenize(text)
    if not tokens:
        return np.zeros(768)  # Return zero vector for empty input

    token_embeddings = np.array([get_roberta_embedding(token) for token in tokens])

    section_tag_emb = get_roberta_embedding(section_tag)
    subsection_tag_emb = get_roberta_embedding(subsection_tag)
    tag_emb = (section_tag_emb + subsection_tag_emb) / 2

    adjusted_embeddings = token_embeddings - tag_emb

    return np.mean(adjusted_embeddings, axis=0)

def process_numerical_data(data, section_tag):
    numerical_text = " ".join(map(str, data))
    return process_subsection(numerical_text, section_tag, "numerical")

def process_row(index):
    start_time = time.time()

    # Extract data
    intro = safe_eval(dataset["Intro"].iloc[index])
    about = safe_eval(dataset["About"].iloc[index])
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process intro data
    if isinstance(intro, dict):
        full_name = intro.get("Full Name", "")
        workplace = intro.get("Workplace", "")
        location = intro.get("Location", "")
    else:
        full_name = workplace = location = str(intro)

    # Process about data
    if isinstance(about, dict):
        desc = about.get("About", "")
    else:
        desc = str(about)

    # Process experiences data
    roles = []
    workplaces_exp = []
    durations = []
    workplace_locations = []
    descriptions = []
    if isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                roles.append(value.get("Role", ""))
                workplaces_exp.append(value.get("Workplace", ""))
                durations.append(value.get("Duration", ""))
                workplace_locations.append(value.get("Workplace Location", ""))
                descriptions.append(value.get("Description", ""))

    # Extract numerical fields
    numerical_fields = [
        "Number of Experiences", "Number of Educations", "Number of Licenses",
        "Number of Volunteering", "Number of Skills", "Number of Recommendations",
        "Number of Projects", "Number of Publications", "Number of Courses",
        "Number of Honors", "Number of Scores", "Number of Languages",
        "Number of Organizations", "Number of Interests", "Number of Activities",
        "Connections", "Followers"
    ]

    numerical_data = []
    for field in numerical_fields:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)  # Default to 0 if conversion fails

    # SSTE Implementation
    subsection_embeddings = []

    # Process Introduction
    subsection_embeddings.append(process_subsection(workplace, "Introduction", "workplace"))
    subsection_embeddings.append(process_subsection(location, "Introduction", "location"))

    # Process About
    subsection_embeddings.append(process_subsection(desc, "About", "description"))

    # Process Experiences
    for role, workplace, duration, location, description in zip(roles, workplaces_exp, durations, workplace_locations, descriptions):
        subsection_embeddings.append(process_subsection(role, "Experiences", "role"))
        subsection_embeddings.append(process_subsection(workplace, "Experiences", "workplace"))
        subsection_embeddings.append(process_subsection(duration, "Experiences", "duration"))
        subsection_embeddings.append(process_subsection(location, "Experiences", "location"))
        subsection_embeddings.append(process_subsection(description, "Experiences", "description"))

    # Process Numerical Data
    subsection_embeddings.append(process_numerical_data(numerical_data, "NumericalData"))

    # Calculate final embedding
    fin = np.mean(subsection_embeddings, axis=0)

    end_time = time.time()
    execution_time = end_time - start_time
    logging.info(f"Processing time for row {index}: {execution_time:.2f} seconds")

    return fin

def generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100):
    all_embeddings = []

    total_start_time = time.time()

    # Check if there's a partially processed file
    try:
        existing_df = pd.read_csv(output_path)
        start_index = len(existing_df)
        all_embeddings = existing_df.values.tolist()
        print(f"Resuming from row {start_index}")
    except FileNotFoundError:
        start_index = 0

    for index in range(start_index, min(num_rows, len(dataset))):
        row_start_time = time.time()
        try:
            embedding = process_row(index)
            all_embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            all_embeddings.append(np.zeros(768))  # RoBERTa embeddings are also 768-dimensional

        row_end_time = time.time()
        row_execution_time = row_end_time - row_start_time
        print(f"Processed row {index + 1} in {row_execution_time:.2f} seconds")

        # Save progress at specified intervals
        if (index + 1) % save_interval == 0:
            temp_df = pd.DataFrame(all_embeddings)
            temp_df.to_csv(output_path, index=False)
            print(f"Progress saved at row {index + 1}")

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save final results to CSV
    csv_start_time = time.time()
    embeddings_df.to_csv(output_path, index=False)
    csv_end_time = time.time()
    csv_execution_time = csv_end_time - csv_start_time
    print(f"Embeddings saved to {output_path} in {csv_execution_time:.2f} seconds")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"Total execution time: {total_execution_time:.2f} seconds")

# Usage
output_path = 'roberta_numerical_SSTE_tokenised_A_NEW.csv'
generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resuming from row 1900
Processed row 1901 in 80.88 seconds


KeyboardInterrupt: 

In [ ]:
import logging
import time
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def process_subsection(text, section_tag, subsection_tag):
    # Get embedding for the entire subsection text
    subsection_embedding = get_roberta_embedding(text)

    # Get tag embeddings
    section_tag_emb = get_roberta_embedding(section_tag)
    subsection_tag_emb = get_roberta_embedding(subsection_tag)

    # Calculate combined tag embedding
    tag_emb = (section_tag_emb + subsection_tag_emb) / 2

    # Adjust subsection embedding with tag embedding
    adjusted_embedding = subsection_embedding - tag_emb

    return adjusted_embedding

def process_row(index):
    start_time = time.time()

    # Extract data
    intro = safe_eval(dataset["Intro"].iloc[index])
    about = safe_eval(dataset["About"].iloc[index])
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process intro data
    if isinstance(intro, dict):
        full_name = intro.get("Full Name", "")
        workplace = intro.get("Workplace", "")
        location = intro.get("Location", "")
    else:
        full_name = workplace = location = str(intro)

    # Process about data
    if isinstance(about, dict):
        desc = about.get("About", "")
    else:
        desc = str(about)

    # Process experiences data
    roles = []
    workplaces_exp = []
    durations = []
    workplace_locations = []
    descriptions = []
    if isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                roles.append(value.get("Role", ""))
                workplaces_exp.append(value.get("Workplace", ""))
                durations.append(value.get("Duration", ""))
                workplace_locations.append(value.get("Workplace Location", ""))
                descriptions.append(value.get("Description", ""))

    # Extract numerical fields
    numerical_fields = [
        "Number of Experiences", "Number of Educations", "Number of Licenses",
        "Number of Volunteering", "Number of Skills", "Number of Recommendations",
        "Number of Projects", "Number of Publications", "Number of Courses",
        "Number of Honors", "Number of Scores", "Number of Languages",
        "Number of Organizations", "Number of Interests", "Number of Activities",
        "Connections", "Followers"
    ]

    numerical_data = []
    for field in numerical_fields:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)

    # SSTE Implementation with section and subsection level processing
    subsection_embeddings = []

    # Process Introduction subsections
    intro_workplace = process_subsection(workplace, "Introduction", "workplace")
    intro_location = process_subsection(location, "Introduction", "location")
    subsection_embeddings.extend([intro_workplace, intro_location])

    # Process About section
    about_emb = process_subsection(desc, "About", "description")
    subsection_embeddings.append(about_emb)

    # Process Experiences subsections
    workplaces_emb = process_subsection(" ".join(workplaces_exp), "Experiences", "workplace")
    roles_emb = process_subsection(" ".join(roles), "Experiences", "role")
    durations_emb = process_subsection(" ".join(durations), "Experiences", "duration")
    locations_emb = process_subsection(" ".join(workplace_locations), "Experiences", "location")
    descriptions_emb = process_subsection(" ".join(descriptions), "Experiences", "description")
    subsection_embeddings.extend([workplaces_emb, roles_emb, durations_emb, locations_emb, descriptions_emb])

    # Process Numerical Data
    numerical_text = " ".join(map(str, numerical_data))
    numerical_emb = process_subsection(numerical_text, "NumericalData", "numerical")
    subsection_embeddings.append(numerical_emb)

    # Calculate final embedding
    fin = np.mean(subsection_embeddings, axis=0)

    end_time = time.time()
    execution_time = end_time - start_time
    logging.info(f"Processing time for row {index}: {execution_time:.2f} seconds")

    return fin

def generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100):
    all_embeddings = []

    total_start_time = time.time()

    # Check if there's a partially processed file
    try:
        existing_df = pd.read_csv(output_path)
        start_index = len(existing_df)
        all_embeddings = existing_df.values.tolist()
        print(f"Resuming from row {start_index}")
    except FileNotFoundError:
        start_index = 0

    for index in range(start_index, min(num_rows, len(dataset))):
        row_start_time = time.time()
        try:
            embedding = process_row(index)
            all_embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            all_embeddings.append(np.zeros(768))  # RoBERTa embeddings are also 768-dimensional

        row_end_time = time.time()
        row_execution_time = row_end_time - row_start_time
        print(f"Processed row {index + 1} in {row_execution_time:.2f} seconds")

        # Save progress at specified intervals
        if (index + 1) % save_interval == 0:
            temp_df = pd.DataFrame(all_embeddings)
            temp_df.to_csv(output_path, index=False)
            print(f"Progress saved at row {index + 1}")

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save final results to CSV
    csv_start_time = time.time()
    embeddings_df.to_csv(output_path, index=False)
    csv_end_time = time.time()
    csv_execution_time = csv_end_time - csv_start_time
    print(f"Embeddings saved to {output_path} in {csv_execution_time:.2f} seconds")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"Total execution time: {total_execution_time:.2f} seconds")

# Usage
output_path = 'roberta_numerical_SSTE_entire_B_NEW.csv'
generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100)

In [ ]:
import logging
import time
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def apply_ste(content_embedding, section_tag):
    tag_embedding = get_roberta_embedding(section_tag)
    return content_embedding - tag_embedding

def process_row(index):
    start_time = time.time()

    # Extract data
    intro = safe_eval(dataset["Intro"].iloc[index])
    about = safe_eval(dataset["About"].iloc[index])
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process intro data
    if isinstance(intro, dict):
        full_name = intro.get("Full Name", "")
        workplace = intro.get("Workplace", "")
        location = intro.get("Location", "")
    else:
        full_name = workplace = location = str(intro)

    # Process about data
    if isinstance(about, dict):
        desc = about.get("About", "")
    else:
        desc = str(about)

    # Process experiences data
    exp_text = ""
    if isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                exp_text += f"{value.get('Role', '')} {value.get('Workplace', '')} {value.get('Duration', '')} {value.get('Workplace Location', '')} {value.get('Description', '')} "

    # Extract numerical fields
    numerical_fields = [
        "Number of Experiences", "Number of Educations", "Number of Licenses",
        "Number of Volunteering", "Number of Skills", "Number of Recommendations",
        "Number of Projects", "Number of Publications", "Number of Courses",
        "Number of Honors", "Number of Scores", "Number of Languages",
        "Number of Organizations", "Number of Interests", "Number of Activities",
        "Connections", "Followers"
    ]

    numerical_data = []
    for field in numerical_fields:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)  # Default to 0 if conversion fails

    # STE Implementation with RoBERTa
    # Intro section
    intro_emb = get_roberta_embedding(f"{full_name} {workplace} {location}")
    fin1 = apply_ste(intro_emb, "Introduction")

    # About section
    about_emb = get_roberta_embedding(desc)
    fin2 = apply_ste(about_emb, "About")

    # Experiences section
    exp_emb = get_roberta_embedding(exp_text)
    fin3 = apply_ste(exp_emb, "Experiences")

    # Numerical data section
    numerical_text = " ".join(map(str, numerical_data))
    numerical_emb = get_roberta_embedding(numerical_text)
    fin4 = apply_ste(numerical_emb, "Numerical data")

    # Combine all embeddings
    fin = (fin1 + fin2 + fin3 + fin4) / 4

    end_time = time.time()
    execution_time = end_time - start_time
    logging.info(f"Processing time for row {index}: {execution_time:.2f} seconds")

    return fin

def generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100):
    all_embeddings = []

    total_start_time = time.time()

    # Check if there's a partially processed file
    try:
        existing_df = pd.read_csv(output_path)
        start_index = len(existing_df)
        all_embeddings = existing_df.values.tolist()
        print(f"Resuming from row {start_index}")
    except FileNotFoundError:
        start_index = 0

    for index in range(start_index, min(num_rows, len(dataset))):
        row_start_time = time.time()
        try:
            embedding = process_row(index)
            all_embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            all_embeddings.append(np.zeros(768))  # RoBERTa embeddings are also 768-dimensional

        row_end_time = time.time()
        row_execution_time = row_end_time - row_start_time
        print(f"Processed row {index + 1} in {row_execution_time:.2f} seconds")

        # Save progress at specified intervals
        if (index + 1) % save_interval == 0:
            temp_df = pd.DataFrame(all_embeddings)
            temp_df.to_csv(output_path, index=False)
            print(f"Progress saved at row {index + 1}")

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save final results to CSV
    csv_start_time = time.time()
    embeddings_df.to_csv(output_path, index=False)
    csv_end_time = time.time()
    csv_execution_time = csv_end_time - csv_start_time
    print(f"Embeddings saved to {output_path} in {csv_execution_time:.2f} seconds")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"Total execution time: {total_execution_time:.2f} seconds")

# Usage
output_path = 'roberta_numerical_STE_entire_C_NEW.csv'
generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100)

In [ ]:
import logging
import time
import ast
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def safe_eval(s):
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def ste_process_section(text, section_tag):
    tokens = tokenizer.tokenize(text)
    if not tokens:
        return np.zeros(768)

    token_embeddings = np.array([get_roberta_embedding(token) for token in tokens])

    section_tag_emb = get_roberta_embedding(section_tag)

    # Adjust token embeddings using only section tag
    adjusted_embeddings = token_embeddings - section_tag_emb

    # Return mean of adjusted token embeddings
    return np.mean(adjusted_embeddings, axis=0)

def process_row(index):
    start_time = time.time()

    # Extract data
    intro = safe_eval(dataset["Intro"].iloc[index])
    about = safe_eval(dataset["About"].iloc[index])
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process intro data
    if isinstance(intro, dict):
        full_name = intro.get("Full Name", "")
        workplace = intro.get("Workplace", "")
        location = intro.get("Location", "")
    else:
        full_name = workplace = location = str(intro)

    # Process about data
    if isinstance(about, dict):
        desc = about.get("About", "")
    else:
        desc = str(about)

    # Process experiences data
    exp_text = ""
    if isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                exp_text += f"{value.get('Role', '')} {value.get('Workplace', '')} {value.get('Duration', '')} {value.get('Workplace Location', '')} {value.get('Description', '')} "

    # Extract numerical fields
    numerical_fields = [
        "Number of Experiences", "Number of Educations", "Number of Licenses",
        "Number of Volunteering", "Number of Skills", "Number of Recommendations",
        "Number of Projects", "Number of Publications", "Number of Courses",
        "Number of Honors", "Number of Scores", "Number of Languages",
        "Number of Organizations", "Number of Interests", "Number of Activities",
        "Connections", "Followers"
    ]

    numerical_data = []
    for field in numerical_fields:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)  # Default to 0 if conversion fails

    # STE Implementation
    section_embeddings = []

    # Process Introduction
    intro_text = f"{full_name} {workplace} {location}"
    section_embeddings.append(ste_process_section(intro_text, "Introduction"))

    # Process About
    section_embeddings.append(ste_process_section(desc, "About"))

    # Process Experiences
    section_embeddings.append(ste_process_section(exp_text, "Experiences"))

    # Process Numerical Data
    numerical_text = " ".join(map(str, numerical_data))
    section_embeddings.append(ste_process_section(numerical_text, "NumericalData"))

    # Calculate final embedding
    fin = np.mean(section_embeddings, axis=0)

    end_time = time.time()
    execution_time = end_time - start_time
    logging.info(f"Processing time for row {index}: {execution_time:.2f} seconds")

    return fin

def generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100):
    all_embeddings = []

    total_start_time = time.time()

    # Check if there's a partially processed file
    try:
        existing_df = pd.read_csv(output_path)
        start_index = len(existing_df)
        all_embeddings = existing_df.values.tolist()
        print(f"Resuming from row {start_index}")
    except FileNotFoundError:
        start_index = 0

    for index in range(start_index, min(num_rows, len(dataset))):
        row_start_time = time.time()
        try:
            embedding = process_row(index)
            all_embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing row {index}: {str(e)}")
            all_embeddings.append(np.zeros(768))  # RoBERTa embeddings are also 768-dimensional

        row_end_time = time.time()
        row_execution_time = row_end_time - row_start_time
        print(f"Processed row {index + 1} in {row_execution_time:.2f} seconds")

        # Save progress at specified intervals
        if (index + 1) % save_interval == 0:
            temp_df = pd.DataFrame(all_embeddings)
            temp_df.to_csv(output_path, index=False)
            print(f"Progress saved at row {index + 1}")

    # Convert list of embeddings to DataFrame
    embeddings_df = pd.DataFrame(all_embeddings)

    # Save final results to CSV
    csv_start_time = time.time()
    embeddings_df.to_csv(output_path, index=False)
    csv_end_time = time.time()
    csv_execution_time = csv_end_time - csv_start_time
    print(f"Embeddings saved to {output_path} in {csv_execution_time:.2f} seconds")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    print(f"Total execution time: {total_execution_time:.2f} seconds")

# Usage
output_path = 'roberta_numerical_STE_tokenised_D_NEW.csv'
generate_embeddings_csv(dataset, output_path, num_rows=3600, save_interval=100)